In [11]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

# 데이터 로드

In [12]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\train.csv"
train = pd.read_csv(file_path)

In [13]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\test.csv"
test = pd.read_csv(file_path)

In [14]:
file_path = r"C:\Users\82106\Desktop\데이콘 한솔데크\data\sample_submission.csv"
sample = pd.read_csv(file_path)

# 전처리

In [16]:
from konlpy.tag import Komoran
import re

komoran = Komoran()

with open('C:/Users/82106/Desktop/데이콘 한솔데크/불용어.txt', 'r', encoding = 'UTF-8') as f:
  list_file = f.readlines() 
stopwords = list_file[0].split(",")

# 정규화
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', ' ', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


# 명사/영단어 추출, 한글자 제외, 불용어 제거
def remove_stopwords(text):
    n = []
    word = komoran.nouns(text)
    p = komoran.pos(text)
    for pos in p:
      if pos[1] in ['SL']:
        word.append(pos[0])
    for w in word:
      if len(w)>1 and w not in stopwords:
        n.append(w)
    return " ".join(n)

# 최종
def finalpreprocess(text):
  return remove_stopwords(preprocess(text))

# 전처리 및 불용어 제거 적용 (두 단계)
train["재발방지대책 및 향후조치계획"] = train["재발방지대책 및 향후조치계획"].apply(finalpreprocess)

# 결과 확인
print(train["재발방지대책 및 향후조치계획"].head())

NameError: name 'string' is not defined

# 모델 로드

In [ ]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

# 인적사고에 따른 재발방지대책 임베딩 및 대표 대책 인덱스 추출

In [ ]:
grouped = train.groupby("인적사고")

res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

# 각 히스토그램에 따른 임베딩 개수

In [ ]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

In [ ]:
res

# 대표 대책 임베딩

In [ ]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [ ]:
sample.info()

# 테스트 데이터에 적용

In [ ]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [ ]:
sample.to_csv("C:/Users/82106/Desktop/데이콘 한솔데크/data/result/baseline.csv", index=False, encoding='utf-8-sig')